In [ ]:
from helper import *

path='../reports/corpus.xlsx'

description = get_column_values(path, 'descriptions_measures', 'A')


targets = get_column_values(path, 'descriptions_measures', 'B')

similarities_df = similarities_df(docs=description,group=targets,encoding_method='tfidf')

similarities_df.head()

In [ ]:
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

path = '../reports/corpus.xlsx'
targets = get_column_values(path, 'descriptions_measures', 'B')
encoding_method = 'word2vec'
version = 'word2vec-google-news-300'




similarities_dict = get_similarity_scores(text=Q1,documents=targets,encoding_method=encoding_method,version=version)

top_10 = get_first_n_keys(similarities_dict, 10)


top_10

In [ ]:
from preprocessing import *
from helper import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '
processed_text = preprocess_text(Q1)



w2v = api.load("word2vec-google-news-300")
vocab = w2v.index_to_key
words = Q1.split()


count = sum(1 for word in processed_text if word in w2v)
existing_words = [word for word in processed_text if word in w2v]
difference = len(words) - count

pr, orig = get_unique_items(words, processed_text)

pr

In [ ]:
from preprocessing import *

Q1 = 'Create a report that shows the number of settlement applications approved during the month, their approved amount,\
      the written off balance, the average days to approval, their average and median duration and their entry principal and balance, \
        for September 2020 per DCA and application type. The report should be produced on Earth portfolio. '

processed_text = preprocess_text(Q1)

processed_text

In [1]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = get_column_values(path, 'descriptions_measures', 'A')
targets = get_column_values(path, 'descriptions_measures', 'B')
models = ['bow','tfidf','word2vec','sbert']
version_w2v = 'word2vec-google-news-300'
version_sbert = 'bert-base-nli-mean-tokens'
version_bow = 'bow_model.joblib'
version_tfidf = 'tfidf_model.joblib'
corpus = ['Descriptions']
tm = TextModels(excel_path=path, columns=corpus,
                 word2vec_version=version_w2v, sbert_model=version_sbert)

#tm.build_bow_model(preprocessing=True)
tm.build_tfidf_model(preprocessing=True)

tm.save_models('preprocessed')

c:\Users\avitsas\Anaconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No bag-of-words model to save.
TF-IDF model saved to preprocessedtfidf_model.joblib


In [2]:
from helper import *

path='../reports/corpus.xlsx'
descriptions = [preprocess_text(element) for element in get_column_values(path, 'descriptions_measures', 'A')]
targets = [preprocess_text(element) for element in get_column_values(path, 'descriptions_measures', 'B')]

targets

['payment prediction stray payer active settlement',
 'application status previous month',
 'payment prediction settlement payer',
 'application type observation period',
 'application type previous observation period',
 'amount paid within observation period',
 'external agency dca legal office assigned previous observation point',
 'status last submitted application within observation period',
 'account status',
 'account segment',
 'qc case code',
 'customer primary account holder',
 'external agency dca legal office currently assigned',
 'portfolio',
 'observation point date',
 'observation point number',
 'principal amount class',
 'asset class',
 'last rejection date application within observation perion',
 'portfolio grouping',
 'customer',
 'unique application identifier',
 'application submission date',
 'approved amount',
 'approved duration',
 'approval date',
 'application origin',
 'observation point number application',
 'observation point date application',
 'application